# Building and Organizing Complex Queries
These are my notes for building complex queries in SQL. I will be using sqlite module along with pandas to run queries. I will be using set operations (union, except, intersect), WITH clauses and different type of joins.

(c) Miradiz Rakhmatov

In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('data/chinook.db')

## This function will help me in generating pandas.DataFrame straight from the sql queries
def run(query):
    return pd.read_sql(query, con)

# Schema diagram:
![](data/chinook-schema.svg)

In [3]:
## database tables

db_content = '''
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table");
'''

run(db_content)

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## 1) Query that shows summary data for every playlist in the Chinook database:
#### Columns to include: playlist ID, playlist name, number of tracks in each playlist, length of each playlist in seconds

In [4]:
q1='''
WITH sub AS 
    (SELECT 
        pl.playlist_id, 
        pl.name, 
        pt.track_id, 
        t.milliseconds/1000 seconds
    FROM playlist pl
    LEFT JOIN playlist_track pt ON pl.playlist_id=pt.playlist_id
    LEFT JOIN track t ON pt.track_id=t.track_id)

SELECT sub.playlist_id playlist_id, sub.name playlist_name, COUNT(sub.track_id) number_of_tracks, SUM(sub.seconds) length_seconds
FROM sub
GROUP BY 1,2
ORDER BY 1 ASC
'''

run(q1)

,playlist_id,playlist_name,number_of_tracks,length_seconds
0,1,Music,3290,876049.0
1,2,Movies,0,NaN
2,3,TV Shows,213,500987.0
3,4,Audiobooks,0,NaN
4,5,90’s Music,1477,397970.0
5,6,Audiobooks,0,NaN
6,7,Movies,0,NaN
7,8,Music,3290,876049.0
8,9,Music Videos,1,294.0
9,10,TV Shows,213,500987.0


## 2) Query to show information on customers that have spent more than 90 dollars in tracks:

In [5]:
q2='''
SELECT c.*
FROM customer c
JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY c.customer_id 
HAVING SUM(i.total) > 90;
'''

## second version by using a subquery

q2v2='''
WITH sub AS (SELECT customer_id, SUM(total)
FROM invoice
GROUP BY 1 
HAVING SUM(total) > 90)

SELECT c.*
FROM customer c
JOIN sub ON sub.customer_id = c.customer_id
'''

run(q2)


,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
6,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
7,21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
8,22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
9,30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


### Let's see if the two query version yield the same results:

In [6]:
print("Is the outcome from both queries the same ?", run(q2v2).equals(run(q2)))

Is the outcome from both queries the same ? True


# Set operations:  

Every SELECT statement within UNION must have the same number of columns.

The columns must also have similar data types.

The columns in every SELECT statement must also be in the same order.



![](data/set_operations.png)

## 3) Query that returns rows where customers have SPENT more than 90 dollars OR/AND ARE from USA. 

* There can be customers that spent MORE than 90 dollars that are from ANY country.
* There can be customers that spent less than 90 dollars that are only from US.

In [7]:
q3='''

SELECT c.*
FROM customer c
JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY c.customer_id 
HAVING SUM(i.total) > 90

UNION

SELECT *
FROM customer
WHERE country = 'USA'
ORDER BY 1
'''

## Using .head() to show only 5 rows
run(q3).head()

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


## 4) This query is almost the same as the above except  it produces DUPLICATE values for the rows with customers that HAVE spent over 90 and ARE from USA

I will use UNION ALL for this query

In [15]:
q4='''

SELECT c.*
FROM customer c
JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY c.customer_id 
HAVING SUM(i.total) > 90

UNION ALL
 
SELECT *
FROM customer
WHERE country = 'USA'

'''
## Using .head() to show only 5 rows
run(q4).head()


,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


### As you can see below there are 4 duplicate values as a result of UNION ALL 

In [9]:
print(run(q4).duplicated(subset=['first_name','last_name']).value_counts())

False    27
True      4
dtype: int64


## 5) Query to show the rows with customers where both statements are TRUE:
## Customers HAVE spent more than 90 dollars and ARE from USA.

In [10]:
q5='''

SELECT c.*
FROM customer c
JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY c.customer_id 
HAVING SUM(i.total) > 90

INTERSECT
 
SELECT *
FROM customer
WHERE country = 'USA'
'''

run(q5)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
1,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
2,21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
3,22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4


In [11]:
## the same result can be achieved with this query

q5v2='''
SELECT c.*
FROM customer c
JOIN invoice i ON i.customer_id = c.customer_id
WHERE country = 'USA'
GROUP BY c.customer_id 
HAVING SUM(i.total) > 90
'''

## 6) Query to show the rows with customers that HAVE spent more than 90 and ARE NOT from USA.

In [12]:
q6='''

SELECT c.*
FROM customer c
JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY c.customer_id 
HAVING SUM(i.total) > 90

EXCEPT
 
SELECT *
FROM customer
WHERE country = 'USA'

'''
run(q6)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3
6,34,João,Fernandes,None,Rua da Assunção 53,Lisbon,None,Portugal,None,+351 (213) 466-111,None,jfernandes@yahoo.pt,4
7,37,Fynn,Zimmermann,None,Berger Straße 10,Frankfurt,None,Germany,60316,+49 069 40598889,None,fzimmermann@yahoo.de,3
8,42,Wyatt,Girard,None,"9, Place Louis Barthou",Bordeaux,None,France,33000,+33 05 56 96 96 96,None,wyatt.girard@yahoo.fr,3
9,46,Hugh,O'Reilly,None,3 Chatham Street,Dublin,Dublin,Ireland,None,+353 01 6792424,None,hughoreilly@apple.ie,3


## 8) Query to show customers from India with their full name and total amount of purchase  

I will be using multiple WITH clauses to define two subqueries

In [13]:
q8='''

WITH 
    india AS(
            SELECT *
            FROM customer
            WHERE country = "India"),
    sales AS(
            SELECT customer_id, SUM(total) total
            FROM invoice
            GROUP BY customer_id)
    
SELECT 
    i.first_name || " " || i.last_name customer_name,  
    s.total total_purchases
FROM india i
JOIN sales s ON i.customer_id=s.customer_id
ORDER BY customer_name
'''

run(q8)

,customer_name,total_purchases
0,Manoj Pareek,111.87
1,Puja Srivastava,71.28


## 9) Query to find the customer from EACH country that has spent the most money at the store

In [14]:
q9='''

WITH sub AS 
            (SELECT c.*, SUM(i.total) total
            FROM customer c 
            JOIN invoice i ON c.customer_id = i.customer_id
            GROUP BY c.customer_id)

SELECT 
    sub.country,
    sub.first_name || " " || sub.last_name customer_name,
    MAX(sub.total) total_purchase
FROM sub
JOIN customer c ON sub.customer_id = c.customer_id
GROUP BY 1

'''
run(q9)

,country,customer_name,total_purchase
0,Argentina,Diego Gutiérrez,39.60
1,Australia,Mark Taylor,81.18
2,Austria,Astrid Gruber,69.30
3,Belgium,Daan Peeters,60.39
4,Brazil,Luís Gonçalves,108.90
5,Canada,François Tremblay,99.99
6,Chile,Luis Rojas,97.02
7,Czech Republic,František Wichterlová,144.54
8,Denmark,Kara Nielsen,37.62
9,Finland,Terhi Hämäläinen,79.20


# THE END